# scribbles
here is a scratchpad for code that isn't quite ready to stand alone in the directories or in it's own script...

In [1]:
# Used to read/manipulate netCDF data
import xarray as xr

# Used to convert to .idx
from OpenVisus import *

# Used for numerical work
import numpy as np

# Used for processing netCDF time data
import datetime

# Used for interacting with OS file system (to get directory file names)
import os

# # Used for redownloading netCDF files from https://firesmoke.ca/ if opening currently downloaded file fails
# import wget

# Accessory, used to generate progress bar for running for loops
from tqdm.notebook import tqdm

In [3]:
# firesmoke_dir = "/Users/arleth/Mount/firesmoke"
firesmoke_dir = "/usr/sci/cedmav/data/firesmoke"

ids = ["BSC18CA12-01", "BSC00CA12-01", "BSC06CA12-01", "BSC12CA12-01"]

In [35]:
file00 = f'{firesmoke_dir}/{ids[1]}/dispersion_20210304.nc'
file18 = f'{firesmoke_dir}/{ids[0]}/dispersion_20210304.nc'

ds00 = xr.open_dataset(file00)
ds18 = xr.open_dataset(file18)

In [36]:
ds00

<xarray.Dataset> Size: 81MB
Dimensions:  (TSTEP: 51, VAR: 1, DATE-TIME: 2, LAY: 1, ROW: 381, COL: 1041)
Dimensions without coordinates: TSTEP, VAR, DATE-TIME, LAY, ROW, COL
Data variables:
    TFLAG    (TSTEP, VAR, DATE-TIME) int32 408B ...
    PM25     (TSTEP, LAY, ROW, COL) float32 81MB ...
Attributes: (12/33)
    IOAPI_VERSION:  $Id: @(#) ioapi library version 3.0 $                    ...
    EXEC_ID:        ????????????????                                         ...
    FTYPE:          1
    CDATE:          2021063
    CTIME:          101914
    WDATE:          2021063
    ...             ...
    VGLVLS:         [10.  0.]
    GDNAM:          HYSPLIT CONC    
    UPNAM:          hysplit2netCDF  
    VAR-LIST:       PM25            
    FILEDESC:       Hysplit Concentration Model Output                       ...
    HISTORY:

In [37]:
ds18

<xarray.Dataset> Size: 71MB
Dimensions:  (TSTEP: 45, VAR: 1, DATE-TIME: 2, LAY: 1, ROW: 381, COL: 1041)
Dimensions without coordinates: TSTEP, VAR, DATE-TIME, LAY, ROW, COL
Data variables:
    TFLAG    (TSTEP, VAR, DATE-TIME) int32 360B ...
    PM25     (TSTEP, LAY, ROW, COL) float32 71MB ...
Attributes: (12/33)
    IOAPI_VERSION:  $Id: @(#) ioapi library version 3.0 $                    ...
    EXEC_ID:        ????????????????                                         ...
    FTYPE:          1
    CDATE:          2021063
    CTIME:          73519
    WDATE:          2021063
    ...             ...
    VGLVLS:         [10.  0.]
    GDNAM:          HYSPLIT CONC    
    UPNAM:          hysplit2netCDF  
    VAR-LIST:       PM25            
    FILEDESC:       Hysplit Concentration Model Output                       ...
    HISTORY:

In [ ]:
ds = xr.open_dataset(f"{tmp_dir}/dispersion.nc")
ds_last = xr.open_dataset(get_latest_file())
ds_CDATE = ds.CDATE
ds_CTIME = ds.CTIME
ds_last_CDATE = ds_last.CDATE
ds_last_CTIME = ds_last.CTIME

# Convert to datetime objects, ensure it's zero-padded to be 6 digits
dt_downloaded = datetime.strptime(f"{ds_CDATE}{ds_CTIME:06}", "%Y%j%H%M%S")
dt_last = datetime.strptime(f"{ds_last_CDATE}{ds_last_CTIME:06}", "%Y%j%H%M%S")

# If dt_last is younger save the dispersion.nc file as 'dispersion_{ds_CDATE}{ds_CTIME}.nc' in final_dir
if dt_last < dt_downloaded:
    ds.to_netcdf(f"{final_dir}/dispersion_{ds_CDATE}{ds_CTIME}")


In [48]:
ds_CDATE = ds00.CDATE
ds_CTIME = ds00.CTIME
ds_last_CDATE = ds18.CDATE
ds_last_CTIME = ds18.CTIME

# Convert to datetime objects, ensure it's zero-padded to be 6 digits
dt_downloaded = datetime.strptime(f"{ds_CDATE}{ds_CTIME:06}", "%Y%j%H%M%S")
dt_last = datetime.strptime(f"{ds_last_CDATE}{ds_last_CTIME:06}", "%Y%j%H%M%S")
print(dt_downloaded)
print(dt_last)
# If dt_last is younger save the dispersion.nc file as 'dispersion_{ds_CDATE}{ds_CTIME}.nc' in final_dir
if dt_last < dt_downloaded:
    print(f"{dt_last} is younger")
    # ds.to_netcdf(f"{final_dir}/dispersion_{ds_CDATE}{ds_CTIME}")
else:
    print(f"{dt_downloaded} is younger")

2021-03-04 10:19:14
2021-03-04 07:35:19
2021-03-04 07:35:19 is younger


In [55]:
print(f"{ds_CDATE}_{ds_CTIME}")

2021063_101914


In [43]:
final_dir = "/usr/sci/cedmav/data/firesmoke/BSC00CA12-01/"

def get_latest_file():
    file_names = sorted(os.listdir(final_dir))
    return file_names[-1]

In [45]:
get_latest_file()

'dispersion_20240209.nc'

In [6]:
# for parsing time flags (TFLAG) from netcdf files
def parse_tflag(tflag):
    year = int(tflag[0] // 1000)
    day_of_year = int(tflag[0] % 1000)
    date = datetime.datetime(year, 1, 1) + datetime.timedelta(days=day_of_year - 1)

    time_in_day = int(tflag[1])
    hours = time_in_day // 10000
    minutes = (time_in_day % 10000) // 100
    seconds = time_in_day % 100

    full_datetime = datetime.datetime(year, date.month, date.day, hours, minutes, seconds)
    return full_datetime

In [25]:
file18 = f'{firesmoke_dir}/{ids[0]}/dispersion_20210304.nc'
file00 = f'{firesmoke_dir}/{ids[1]}/dispersion_20210304.nc'
file06 = f'{firesmoke_dir}/{ids[2]}/dispersion_20210304.nc'
file12 = f'{firesmoke_dir}/{ids[3]}/dispersion_20210304.nc'

file_mine = f'{firesmoke_dir}/{ids[1]}/dispersion_20240209.nc'

In [26]:
ds00 = xr.open_dataset(file00)
ds06 = xr.open_dataset(file06)
ds12 = xr.open_dataset(file12)
ds18 = xr.open_dataset(file18)

ds_mine = xr.open_dataset(file_mine)

In [31]:
type(ds18.CTIME)

numpy.int32

In [34]:
# Convert to string and parse using datetime.strptime
time_obj = datetime.strptime(f"{234}", "%H%M%S")
# Format as HH:MM:SS
formatted_time = time_obj.strftime("%H:%M:%S")

print(formatted_time)

02:03:04


In [32]:
# Convert to string and parse using datetime.strptime
time_obj = datetime.strptime(f"{4335}", "%H%M%S")
# Format as HH:MM:SS
formatted_time = time_obj.strftime("%H:%M:%S")

print(formatted_time)

04:33:05


In [ ]:
time18 = [parse_tflag(t).strftime("%Y-%m-%d %H:%M:%S") for t in np.squeeze(ds18['TFLAG'].values)]
time18[0:6]

In [ ]:
time00 = [parse_tflag(t).strftime("%Y-%m-%d %H:%M:%S") for t in np.squeeze(ds00['TFLAG'].values)]
time00[0:6]

In [ ]:
time06 = [parse_tflag(t).strftime("%Y-%m-%d %H:%M:%S") for t in np.squeeze(ds06['TFLAG'].values)]
time06[0:6]

In [ ]:
time12 = [parse_tflag(t).strftime("%Y-%m-%d %H:%M:%S") for t in np.squeeze(ds12['TFLAG'].values)]
time12[0:6]

In [ ]:
print(f'time00: {time00[0]} to {time00[-1]}')
print(f'time06: {time06[0]} to {time06[-1]}')
print(f'time12: {time12[0]} to {time12[-1]}')
print(f'time18: {time18[0]} to {time18[-1]}')

In [ ]:
print(f"time00: {datetime.datetime.strptime(time00[0], '%Y-%m-%d %H:%M:%S') - datetime.datetime.strptime(time00[-1], '%Y-%m-%d %H:%M:%S')}")
print(f"time06: {datetime.datetime.strptime(time06[0], '%Y-%m-%d %H:%M:%S') - datetime.datetime.strptime(time06[-1], '%Y-%m-%d %H:%M:%S')}")
print(f"time12: {datetime.datetime.strptime(time12[0], '%Y-%m-%d %H:%M:%S') - datetime.datetime.strptime(time12[-1], '%Y-%m-%d %H:%M:%S')}")
print(f"time18: {datetime.datetime.strptime(time18[0], '%Y-%m-%d %H:%M:%S') - datetime.datetime.strptime(time18[-1], '%Y-%m-%d %H:%M:%S')}")

In [ ]:
print(f"file00: {file00}")
print(f"file06: {file06}")
print(f"file12: {file12}")
print(f"file18: {file18}")

In [ ]:
import pickle

# Load conversion_seq from a file
with open('conversion/idx_calls_v4.pkl', 'rb') as f:
    idx_calls_v4 = pickle.load(f)

# Load idx_calls from a file
with open('conversion/idx_calls_v3.pkl', 'rb') as f:
    idx_calls_v3 = pickle.load(f)

---

Western Canada `BSC00WC04-01` dataset scribbles

In [14]:
firesmoke_dir = "/usr/sci/scratch_nvme/arleth/download/BSC00WC04-01"
datasets = []
times = []
for i in np.arange(1, 5):
    datasets.append(xr.open_dataset(f'{firesmoke_dir}/dispersion_2022011{i}.nc'))

for ds in datasets:
    t = [parse_tflag(t).strftime("%Y-%m-%d %H:%M:%S") for t in np.squeeze(ds['TFLAG'].values)]
    times.append(t)

In [18]:
# print time range for each file
for t in times:
    print(f"t[0]: {datetime.datetime.strptime(t[0], '%Y-%m-%d %H:%M:%S') - datetime.datetime.strptime(t[-1], '%Y-%m-%d %H:%M:%S')}")

t[0]: -3 days, 22:00:00
t[0]: -3 days, 22:00:00
t[0]: -3 days, 22:00:00
t[0]: -3 days, 22:00:00
